In [15]:
from scipy.ndimage import generate_binary_structure, binary_erosion
import SimpleITK as sitk
import numpy as np

# define metrics

In [2]:
def get_accuracy(ground_truth, predicted):

    true_positives = np.sum(np.logical_and(ground_truth, predicted))
    true_negatives = np.sum(np.logical_and(1-ground_truth, 1-predicted))
    false_positives = np.sum(np.logical_and(1-ground_truth, predicted))
    false_nagatives = np.sum(np.logical_and(ground_truth, 1-predicted))

    return (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_nagatives)

In [3]:
def get_precision(ground_truth, predicted):

    true_positives = np.sum(np.logical_and(ground_truth, predicted))
    false_positives = np.sum(np.logical_and(1-ground_truth, predicted))

    return (true_positives) / (true_positives + false_positives)

In [4]:
def get_recall(ground_truth, predicted):

    true_positives = np.sum(np.logical_and(ground_truth, predicted))
    false_nagatives = np.sum(np.logical_and(ground_truth, 1-predicted))

    return (true_positives) / (true_positives + false_nagatives)

In [5]:
def get_specificity(ground_truth, predicted):
    true_negatives = np.sum(np.logical_and(1-ground_truth, 1-predicted))
    false_positives = np.sum(np.logical_and(1-ground_truth, predicted))

    return (true_negatives) / (true_negatives + false_positives)

In [6]:
def get_f1_score(ground_truth, predicted):

    precision = get_precision(ground_truth, predicted)
    recall = get_recall(ground_truth, predicted)

    return 2 * (precision * recall) / (precision + recall)

In [35]:
def get_Jaccard(ground_truth, predicted):
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(sitk.GetImageFromArray(ground_truth), sitk.GetImageFromArray(predicted))

    return overlap_measures_filter.GetJaccardCoefficient()

# # Overwritten with equally valid computation 'by hand' (not using black-box function from SimpleITK library)
# def get_Jaccard1(ground_truth, predicted):
#     true_positives = np.sum(np.logical_and(ground_truth, predicted))
#     false_positives = np.sum(np.logical_and(1-ground_truth, predicted))
#     false_nagatives = np.sum(np.logical_and(ground_truth, 1-predicted))

#     return 1 - abs(false_nagatives - false_positives) / (2 * true_positives + false_positives + false_nagatives)

# print(get_Jaccard(ground_truth, predictions[0]))
# print(get_Jaccard1(ground_truth, predictions[0]))

# print(get_Jaccard(ground_truth, predictions[1]))
# print(get_Jaccard1(ground_truth, predictions[1]))

0.972972972972973
0.9863013698630136
0.9459459459459459
0.9722222222222222


In [8]:
def get_DSC(ground_truth, predicted):
    return get_f1_score(ground_truth, predicted)

In [9]:
def get_volume_similarity(ground_truth, predicted):
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(sitk.GetImageFromArray(ground_truth), sitk.GetImageFromArray(predicted))
    
    return overlap_measures_filter.GetVolumeSimilarity()

def get_volume_similarity(ground_truth, predicted):
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(sitk.GetImageFromArray(ground_truth), sitk.GetImageFromArray(predicted))
    
    return overlap_measures_filter.GetVolumeSimilarity()

In [31]:
def get_HD(ground_truth, predicted):
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(sitk.GetImageFromArray(ground_truth), sitk.GetImageFromArray(predicted))
    
    return hausdorff_distance_filter.GetHausdorffDistance()

def get_HD(ground_truth, predicted):
    conn = generate_binary_structure(ground_truth.ndim, connectivity=1)

    S = ground_truth - binary_erosion(ground_truth, conn)
    Sprime = predicted - binary_erosion(predicted, conn)

    # Get coordinates of nonzero elements
    A_coords = np.transpose(np.nonzero(S))
    B_coords = np.transpose(np.nonzero(Sprime))

    # Compute the max min distance from each point
    h_AB = np.max([np.min(np.linalg.norm(a - B_coords, axis=1)) for a in A_coords])
    h_BA = np.max([np.min(np.linalg.norm(b - A_coords, axis=1)) for b in B_coords])

    return max(h_AB, h_BA)

In [11]:
def get_SSASD(input1, input2):

    conn = generate_binary_structure(input1.ndim, connectivity=1)

    S = input1 - binary_erosion(input1, conn)
    Sprime = input2 - binary_erosion(input2, conn)

    # Get coordinates of nonzero elements
    A_coords = np.transpose(np.nonzero(S))
    B_coords = np.transpose(np.nonzero(Sprime))

    # Compute the average of minimum distances from A to B
    d_AB = np.mean([np.min(np.linalg.norm(a - B_coords, axis=1)) for a in A_coords])

    # Compute the average of minimum distances from B to A
    d_BA = np.mean([np.min(np.linalg.norm(b - A_coords, axis=1)) for b in B_coords])

    # Compute the surface symmetric average surface distance
    SSASD = (d_AB + d_BA) / 2

    return SSASD

# Execute metrics on examples

In [17]:
ground_truth = np.array([[0,0,0,0,0,0,0,0,0,0,0],
                         [0,0,0,0,0,0,0,0,0,0,0],
                         [0,0,1,1,1,1,1,1,1,0,0],
                         [0,0,1,1,1,1,1,1,1,0,0],
                         [0,0,1,1,1,1,1,1,1,0,0],
                         [0,0,1,1,1,1,0,0,0,0,0],
                         [0,0,1,1,1,1,0,0,0,0,0],
                         [0,0,1,1,1,1,0,0,0,0,0],
                         [0,0,1,1,1,1,0,0,0,0,0],
                         [0,0,0,0,0,0,0,0,0,0,0],
                         [0,0,0,0,0,0,0,0,0,0,0]])

predictions = [
    np.array([[0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0],
              [0,0,1,1,1,1,1,1,1,0,0],
              [0,0,1,1,1,1,1,1,1,0,0],
              [0,0,1,1,1,1,1,1,1,0,0],
              [0,0,1,1,1,1,0,0,0,0,0],
              [0,0,1,1,1,1,0,0,0,0,0],
              [0,0,1,1,1,1,0,0,0,0,0],
              [0,0,0,1,1,1,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0]]),

    np.array([[0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0],
              [0,0,1,1,1,1,1,1,1,0,0],
              [0,0,1,1,1,1,1,1,1,0,0],
              [0,0,1,1,1,1,1,1,1,0,0],
              [0,0,1,1,1,1,0,0,0,0,0],
              [0,0,1,1,1,1,0,0,0,0,0],
              [0,0,0,1,1,1,0,0,0,0,0],
              [0,0,0,1,1,1,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0,0,0]]),
]
                       

In [14]:
for prediction in predictions:
    print("Accuracy: ", get_accuracy(ground_truth, prediction))
    print("Precision: ", get_precision(ground_truth, prediction))
    print("Recall: ", get_recall(ground_truth, prediction))
    print("Specificity: ", get_specificity(ground_truth, prediction))
    print("F1 Score: ", get_f1_score(ground_truth, prediction))
    print("Jaccard: ", get_Jaccard(ground_truth, prediction))
    print("DSC: ", get_DSC(ground_truth, prediction))
    print("Volume Similarity: ", get_volume_similarity(ground_truth, prediction))
    print("Hausdorff Distance: ", get_HD(ground_truth, prediction))
    print("Average Symmetric Surface Distance: ", get_SSASD(ground_truth, prediction))
    print("\n")

Accuracy:  0.9917355371900827
Precision:  1.0
Recall:  0.972972972972973
Specificity:  1.0
F1 Score:  0.9863013698630138
Jaccard:  0.972972972972973
DSC:  0.9863013698630138
Volume Similarity:  0.0273972602739726
Hausdorff Distance:  1.0
Average Symmetric Surface Distance:  0.021739130434782608


Accuracy:  0.9834710743801653
Precision:  1.0
Recall:  0.9459459459459459
Specificity:  1.0
F1 Score:  0.9722222222222222
Jaccard:  0.9459459459459459
DSC:  0.9722222222222222
Volume Similarity:  0.05555555555555555
Hausdorff Distance:  1.0
Average Symmetric Surface Distance:  0.06620553359683795


